In [1]:
%load_ext ngql
%ngql --address 127.0.0.1 --port 9669 --user root --password nebula

Connection Pool Created


,Name
0,chinese_kg
1,gov_graph1
2,guardians
3,session_pool_test
4,test_kg1
5,test_kg2
6,test_kg3


In [2]:
endpoint_url = "http://127.0.0.1:8000" # LLM API
embedding_model = "/home/cuijc/lsp/text2vec-base-chinese"

import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO) # logging.DEBUG for more verbose output
# logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))


# LLM
from langchain.llms import ChatGLM
from langchain import PromptTemplate, LLMChain


llm = ChatGLM(
    endpoint_url=endpoint_url,
    max_token=2048,
    top_p=0.9,
    temperature=1,
    model_kwargs={
        "sample_model_args": False,
    },
)

llm.with_history = False

In [3]:
import torch.cuda
import torch.backends

EMBEDDING_DEVICE = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"

from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index import LangchainEmbedding

embed_model = LangchainEmbedding(
  HuggingFaceEmbeddings(
      model_name=embedding_model,
      model_kwargs={'device': EMBEDDING_DEVICE},
  )
)

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: /home/cuijc/lsp/text2vec-base-chinese


In [4]:
# Llama Index ServiceContext

from llama_index import ServiceContext, LLMPredictor

llm_predictor = LLMPredictor(llm=llm)

service_context = ServiceContext.from_defaults(
    llm_predictor=llm_predictor,
    embed_model=embed_model,
)

# Set global service context

from llama_index import set_global_service_context

set_global_service_context(service_context)

In [5]:
%%ngql
CREATE SPACE IF NOT EXISTS gov_graph1(vid_type=FIXED_STRING(256), partition_num=1, replica_factor=1);
USE gov_graph1;
CREATE TAG IF NOT EXISTS entity(name string);
CREATE EDGE IF NOT EXISTS relationship(relationship string);
CREATE TAG INDEX IF NOT EXISTS entity_index ON entity(name(256));

INFO:nebula3.logger:Get connection to ('127.0.0.1', 9669)


""


In [6]:
import os

os.environ['NEBULA_USER'] = "root"
os.environ['NEBULA_PASSWORD'] = "nebula"
os.environ['NEBULA_ADDRESS'] = "127.0.0.1:9669"

space_name = "gov_graph1"
edge_types, rel_prop_names = ["relationship"], ["relationship"]
tags = ["entity"]

from llama_index.storage.storage_context import StorageContext
from llama_index.graph_stores import NebulaGraphStore

graph_store = NebulaGraphStore(
    space_name=space_name,
    edge_types=edge_types,
    rel_prop_names=rel_prop_names,
    tags=tags)

storage_context = StorageContext.from_defaults(graph_store=graph_store)

In [7]:
import os
from llama_index import SimpleDirectoryReader

# 本地目录路径
local_directory_path = "/home/cuijc/lsp/nebula_graph/gov_report1"

for root, dirs, files in os.walk(local_directory_path):
    for file in files:
        if file.endswith(".md"):
            os.rename(os.path.join(root, file), os.path.join(root, file.replace(".md", ".txt")))

# 从指定的本地目录加载文件
# documents = SimpleDirectoryReader(local_directory_path, exclude_hidden=True).load_data()
documents = SimpleDirectoryReader(local_directory_path, recursive=True, exclude_hidden=True).load_data()

print(f'Loaded {len(documents)} documents')

Loaded 1 documents


In [8]:
from llama_index.prompts.base import Prompt
from llama_index.prompts.prompt_type import PromptType

KG_TRIPLET_EXTRACT_TMPL = """
根据给定的文本，通过一步一步总结，理解，最终输出抽取至多 {max_knowledge_triplets} 行 (主语, 谓语, 宾语) 格式的三元组用作构建问答知识图谱，忽略文本中的停止符号。
<注意> 保证三元组只有主谓宾三部分，不要把罗列的知识放在一行中，而应该拆为多行知识。
<注意> 如果文本是大段代码或者命令行、罗列的步骤，先总结出知识再抽取有意义的知识。
<注意> 返回格式为每一行用括号包裹、逗号隔开，没有序号。
<注意> 仔细检查，只抽取有意义的知识，没有的时候返回空。
<注意> 谓语要翻译成中文。
<注意> 要注意三元组主语的选择，要明确，准确。

下面是几个例子:
---------------------
文本: 狗是人类最早驯化的动物，大约在一万四千年前，人类就开始驯化狼，最终演化成了我们现在看到的各种犬种。狗属于哺乳动物，其视觉、听觉和嗅觉都非常灵敏。它们是社会性的动物，通常在群体中生活。狗的寿命一般在10到15年之间，但也有一些犬种可以活到20年以上。它们的食物主要是肉类，但是也能吃一些蔬菜和谷物。
主谓宾三元组:
(狗, 是, 人类最早驯化的动物)
(狗, 属于, 哺乳动物)
(狗, 寿命为, 10到15年之间)
----
# 本例中只抽取不超过 5 行三元组，且将列表信息综合处理。
文本: Docker 的安装
Docker 是一个开源的商业产品，有两个版本：社区版（Community Edition，缩写为 CE）和企业版（Enterprise Edition，缩写为 EE）。企业版包含了一些收费服务，个人开发者一般用不到。下面的介绍都针对社区版。

Docker CE 的安装请参考官方文档，并且支持：

- Mac
- Windows
- Ubuntu
- Debian
- CentOS
- Fedora

对于其他 Linux 发行版
安装完成后，运行下面的命令，验证是否安装成功。


$ docker version
# 或者
$ docker info
Docker 需要用户具有 sudo 权限，为了避免每次命令都输入sudo，可以把用户加入 Docker 用户组（官方文档）。


$ sudo usermod -aG docker $USER
Docker 是服务器----客户端架构。命令行运行docker命令的时候，需要本机有 Docker 服务。如果这项服务没有启动，可以用下面的命令启动（官方文档）。


# service 命令的用法
$ sudo service docker start

# systemctl 命令的用法
$ sudo systemctl start docker
主谓宾三元组:
(Docker, 是, 开源的商业产品)
(Docker CE, 支持, Mac、Windows 和 Linux)
(Docker, 包含, 一些收费服务)
(Docker, 需要, 用户具有sudo权限)
(Docker, 是, 服务器-客户端架构)
---------------------

下面请根据之前的要求和例子，开始知识抽取任务！
---------------------
文本: {text}


主谓宾三元组:
"""

KG_TRIPLET_EXTRACT_PROMPT = Prompt(
    KG_TRIPLET_EXTRACT_TMPL, prompt_type=PromptType.KNOWLEDGE_TRIPLET_EXTRACT
)

QUERY_KEYWORD_EXTRACT_TEMPLATE_TMPL = (
    "根据下列要求完成任务，不要忘记 <注意> 的要求\n"
    "根据给定的文本，抽取不超过 {max_keywords} 个实体名词关键词，"
    "这些关键词是作为适合在知识图谱中进行查询的实体。忽略文本中的停止符号。\n"
    "<注意> 如果有英文，给出多种合理的大小写情况的关键词，比如关键词 Baseball park，"
    "可能要给出 'KEYWORDS: Baseball park, Baseball Park'\n"
    "<注意> 不要超出 {max_keywords} 个关键词\n"
    "<注意> 只返回要求的 KEYWORDS: 开头，然后英文逗号隔开的格式，不带序号、换行。\n"
    "---------------------\n"
    "{question}\n"
    "---------------------\n"
    "现在返回其中可能得关键词，以这样的格式 --> 'KEYWORDS: keyword1, keyword2, keyword3'\n"
)
QUERY_KEYWORD_EXTRACT_TEMPLATE = Prompt(
    QUERY_KEYWORD_EXTRACT_TEMPLATE_TMPL,
    prompt_type=PromptType.QUERY_KEYWORD_EXTRACT,
)

In [10]:
from llama_index import KnowledgeGraphIndex

graph_store.query("SHOW HOSTS")

kg_index = KnowledgeGraphIndex.from_documents(
    documents,
    storage_context=storage_context,
    max_triplets_per_chunk=5,
    service_context=service_context,
    space_name=space_name,
    edge_types=edge_types,
    rel_prop_names=rel_prop_names,
    tags=tags,
    kg_triple_extract_template=KG_TRIPLET_EXTRACT_PROMPT,
    query_keyword_extract_template=QUERY_KEYWORD_EXTRACT_TEMPLATE,
    max_knowledge_sequence=15,
)

kg_index.storage_context.persist(persist_dir='nebula_graph/storage_graph3')
!ls -l nebula_graph/storage_graph3

(王伟中, 代表省人民政府, 向大会报告工作)
(向大会报告工作, 是不, 广东省政府)
(过去的五年, 极不寻常、极不平凡, 是, 广东省人民政府)
(一、过去五年工作回顾, 开始, 广东省政府)
(一、过去的五年工作回顾, 是不, 广东省政府)
(一、过去的五年工作回顾, 主语, 王伟中)
(一、过去的五年工作回顾, 宾语, 向大会报告工作)
(广东省, 是, 五年来坚持稳中求进工作总基调)
(广东省, 完整、准确、全面贯彻新发展理念)
(广东省, 扎实打造新发展格局战略支点)
(广东省, 经济实力实现大步跨越)
(广东省, 预计2022年全省地区生产总值达12.8万亿元)
(广东省, 五年跨过3个万亿元级台阶、年均增长5%)
(广东省, 连续34年居全国首位)
(广东省, 地方一般公共预算收入达1.33万亿元)
(广东省, 外贸进出口总额达8.3万亿元)
(广东省, 市场主体总量突破1600万户、五年净增608万户)
(广东省, 其中企业超过700万户、占全国1/7)
(广东省, 进入世界500强企业达17家、五年增加6家)
(广东省, 三次产业比重调整为4.2∶41.1∶54.7)
(广东省, 先进制造业和高技术制造业增加值占规模以上工业比重分别提高到55%、29.5%)
(广东省, 金融业增加值达1.15万亿元)
(广东省, 现代服务业增加值占服务业比重达65.9%)
(广东省, 坚决落实重大国家战略)
(广东省, 抢抓机遇推进“双区”和横琴、前海、南沙三大平台建设)
(广东省, 改革开放迸发强劲活力)
(广东省, 《粤港澳大湾区发展规划纲要》顺利实施)
(广东省, 港珠澳大桥、广深港高铁等标志性工程建成通车)
(广东省, “湾区通”工程深入实施)
(广东省, 大湾区国际科技创新中心建设加快推进)
(广东省, 国际一流湾区和世界级城市群展现蓬勃活力)
(深圳, 着力推进, 高水平科技自立自强)
(深圳, 全力打造, 具有全球影响力的科技和产业创新高地)
(深圳, 高质量发展, 迈出新步伐)
(深圳, 实施制造业高质量发展, "六大工程")
(深圳, 高起点培育, 20个战略性产业集群)
(深圳, 形成, 新一代电子信息、绿色石化、智能家电、先进材料、现代轻工纺织、软件与信息服务、现代农业与食品、汽车等8个万亿元级产业集群)
(深圳, 规模以上工业企业达, 

In [9]:
from llama_index import load_index_from_storage

storage_context_graph = StorageContext.from_defaults(persist_dir='nebula_graph/storage_graph3', graph_store=graph_store)

kg_index = load_index_from_storage(
    storage_context=storage_context_graph,
    max_triplets_per_chunk=5,
    service_context=service_context,
    space_name=space_name,
    edge_types=edge_types,
    rel_prop_names=rel_prop_names,
    tags=tags,
    kg_triple_extract_template=KG_TRIPLET_EXTRACT_PROMPT,
    query_keyword_extract_template=QUERY_KEYWORD_EXTRACT_TEMPLATE,
    max_knowledge_sequence=15,
)

INFO:llama_index.indices.loading:Loading all indices.


In [12]:
%ngql USE gov_graph1;
%ngql MATCH ()-[e]->() RETURN e

INFO:nebula3.logger:Get connection to ('127.0.0.1', 9669)
INFO:nebula3.logger:Get connection to ('127.0.0.1', 9669)


,e
0,"(""重要生态区域"")-[:relationship@-3454155533719776787..."
1,"(""蓝天、碧水、净土保卫战"")-[:relationship@-48798739458092..."
2,"(""菠萝"")-[:relationship@-6164880685669392041{rel..."
3,"(""荔枝"")-[:relationship@-6164880685669392041{rel..."
4,"(""粤贸全球"")-[:relationship@1047091911530513736{re..."
...,...
336,"(""17"")-[:relationship@4614547196859289288{rela..."
337,"(""17"")-[:relationship@4614547196859289288{rela..."
338,"(""17"")-[:relationship@4614547196859289288{rela..."
339,"(""12355青少年服务热线"")-[:relationship@66255431195444..."


In [13]:
%ng_draw

<class 'pyvis.network.Network'> |N|=409 |E|=341

In [10]:
from llama_index import VectorStoreIndex

vector_index = VectorStoreIndex.from_documents(
    documents,
    service_context=service_context,
)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [11]:
vector_query_engine = vector_index.as_query_engine()

kg_keyword_query_engine = kg_index.as_query_engine(
    include_text=False,
    retriever_mode="keyword",
    # max_keywords_per_query=3,
    max_keywords_per_query=10,
)

In [14]:
from IPython.display import Markdown, display

In [16]:
response = vector_query_engine.query("请总结过去五年来的工作，中文回答")
display(Markdown(f"<b>{response}</b>"))

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

<b>根据广东省省长王伟中在2023年1月12日广东省第十四届人民代表大会第一次会议上的讲话报告，过去五年广东省取得了来之不易的成绩，经验弥足珍贵。在这一过程中，以习近平同志为核心的党中央坚强领导、亲切关怀是不可或缺的。同时，广东省委带领全省人民一道拼出来、干出来、奋斗出来，充分展现了各省的团结合作和努力奋斗精神。</b>

In [24]:
response_graph = kg_keyword_query_engine.query("请总结过去五年来的工作，中文回答")
display(Markdown(f"<b>{response_graph}</b>"))

<b>在过去五年中，我国在多个领域取得了显著的成绩和进展。以下是其中几个方面的总结：

1. 高等教育：毛入学率提高至57%以上，职业教育在校生规模居全国前列，新增基础教育公办学位252万个。
2. 城镇就业：新增就业690万人、占全国1/10，城镇新增就业人数逐年增长。
3. 餐饮服务：粤菜师傅、广东技工、南粤家政三项工程培训893万人次。
4. 收入水平：居民人均可支配收入超过4.5万元、增速与经济增长基本同步，人民生活质量不断提高。
5. 社会发展：坚定践行以人民为中心的发展思想，人民群众获得感幸福感安全感进一步提升。
6. 民生实事：坚持“小切口大变化”办好民生实事，解决人民群众的实际需求。</b>

In [39]:
response = vector_query_engine.query("请总结有关深圳的工作，详细回答")
display(Markdown(f"<b>{response}</b>"))

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

<b>深圳在过去五年中取得了显著的工作成果。在高水平科技自立自强的进程中，深圳成功落地首批40项综合改革试点，并在18条试点经验上全国推广。同时，深圳在前海深港现代服务业合作区、横琴粤澳深度合作区、南沙面向世界的粤港澳全面合作方面也取得了良好的开局。在优化营商环境、数字政府、要素市场化、国资国企等改革方面，深圳走在全国前列。此外，五年实际利用外资超过8000亿元，引进了百亿美元级项目，充分彰显广东的"两个重要窗口"作用。

在制造业高质量发展方面，深圳实施制造业高质量发展"六大工程"，高起点培育20个战略性产业集群，形成新一代电子信息、绿色石化、智能家电、先进材料、现代轻工纺织、软件与信息服务、现代农业与食品、汽车等8个万亿元级产业集群。规模以上工业企业达6.7万家，五年增加2万家，高新技术企业达6.9万家，五年翻了一番多。

此外，深圳在2020年成功举办了服贸会，并举办了深圳国际马拉松等大型活动，吸引了来自全球各地的关注。总的来说，深圳在过去五年中取得了显著的工作成果，为中国的现代化建设作出了积极贡献。</b>

In [38]:
response_graph = kg_keyword_query_engine.query("请总结有关深圳的工作，详细回答")
display(Markdown(f"<b>{response_graph}</b>"))

<b>深圳是中国广东省的一个辖地级市，位于珠江三角洲南部。深圳是中国改革开放的重要窗口和经济特区之一，也是中国特色社会主义先行示范区的重点城市。深圳的经济发展模式以电子信息产业为主导，坚持创新、协调、绿色、开放、共享的新发展理念，推进高质量发展、高水平开放、高水平人才队伍建设，致力于打造具有全球影响力的科技和产业创新高地。

深圳的工作主要集中在以下几个方面：

1. 经济发展：深圳以电子信息产业为主导，推进产业结构升级和创新创业，打造国际先进制造业和创新中心。深圳的GDP居全国前列，是中国经济特区的佼佼者。

2. 改革开放：深圳是中国改革开放的重要窗口，积极发挥先行示范区的作用，深入推进改革开放，加强国际科技、经济、文化等方面的合作与交流。

3. 科技创新：深圳注重科技创新，建立了一批国家重点实验室和工程技术研究中心，拥有一批高科技企业，如华为、腾讯等。深圳还积极引进国际先进科技和人才，促进科技创新和人才培养。

4. 文化建设：深圳文化建设取得显著成果，建设了一批文化场馆和艺术机构，如深圳音乐厅、深圳美术馆等。此外，深圳也在传承和发展中华优秀传统文化方面做出了积极贡献。

5. 民生建设：深圳注重民生建设，提高了社会福利水平，如建设了优质学校、医院等公共服务设施，提供了优美的生活环境。

6. 城市建设：深圳的城市建设取得显著成果，建成了现代化城市面貌，如深圳湾超级总部基地、香蜜湖新金融中心等城市重点片区。

7. 社会治理：深圳重视社会治理，推进了数字化治理、综治信息化等工作，完善了社会管理服务体系，确保了社会安定和谐。

综上所述，深圳作为一个具有先行示范区优势的城市，充分发挥自身优势，积极推进经济、文化、社会等领域的创新和发展，努力提高市民生活水平，为国家和地区的经济社会发展做出了积极贡献。</b>

In [40]:
response = vector_query_engine.query("请总结有关改革的工作，详细回答")
display(Markdown(f"<b>{response}</b>"))

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

<b>The passage provides an overview of the work being done in reform and development in the Guangzhou City. It focuses on various areas such as reform,扩大开放,发展平衡性协调性,以及推动高质量发展。在改革方面,文章提到了实施高标准市场准入制度、行政许可制度、落实企业主体责任、完善要素市场化配置体制机制、健全金融服务体制机制、打造法治化营商环境、提高社会文明程度等方面的努力。在扩大开放方面,文章提到了加快构建高水平社会主义市场经济体制、实施“五外联动”组合拳、加快建设贸易强省、推动珠三角产业向粤东粤西粤北地区有序转移等方面的措施。在发展平衡性协调性方面,文章提到了实施乡村振兴战略、区域协调发展战略、主体功能区战略、新型城镇化战略,推进强县促镇带村,加快汕头、湛江省域副中心城市建设,推动产业转移等措施。最后,文章提到了在改革方面,实施制造业高质量发展“六大工程”,形成新一代电子信息、绿色石化、智能家电、先进材料、现代轻工纺织、软件与信息服务、现代农业与食品、汽车等8个万亿元级产业集群,以及规模以上工业企业达6.7万家、五年增加2万家,高新技术企业达6.9万家、五年翻了一番多等措施。</b>

In [16]:
response_graph = kg_keyword_query_engine.query("请总结有关深圳的工作，详细回答")
display(Markdown(f"<b>{response_graph}</b>"))

<b>深圳是中国广东省的一个城市，是中国改革开放的重要窗口和经济特区之一。以下是深圳的一些工作总结：

深圳的主要工作是推动经济高质量发展，助力中国特色社会主义先行示范区建设。深圳的经济发展高度依赖出口，特别是电子信息产业。深圳市政府积极引进外资，加强与世界各国的科技合作与交流，推动产业结构升级和经济发展。

同时，深圳也注重科技创新和人才发展。深圳市政府制定了一系列相关政策，鼓励企业增加研发投入，推动科技创新。此外，深圳还在教育、医疗等领域加强投入，提高人民的综合素质。

此外，深圳积极参与国际合作和交流，加强与外国城市的交流合作，促进国际经贸关系的发展。

总的来说，深圳市政府在推动经济高质量发展、助力先行示范区建设、加强科技创新和国际合作等方面，都在取得积极的成绩。</b>

In [17]:
list(response_graph.metadata.values())[0]['kg_rel_map']

{'深圳{name: 深圳}': ['深圳{name: 深圳} -[relationship:{relationship: 是}]-> 先行示范区{name: 先行示范区} <-[relationship:{relationship: 建设}]- 深圳{name: 深圳}',
  '深圳{name: 深圳} -[relationship:{relationship: 建设}]-> 先行示范区{name: 先行示范区}',
  '深圳{name: 深圳} -[relationship:{relationship: 实施制造业高质量发展}]-> 六大工程{name: 六大工程}',
  '深圳{name: 深圳} -[relationship:{relationship: 五年增加}]-> 2万家{name: 2万家}',
  '深圳{name: 深圳} -[relationship:{relationship: 加快编制实施}]-> 横琴合作区总体发展规划{name: 横琴合作区总体发展规划}',
  '深圳{name: 深圳} -[relationship:{relationship: 先行示范区建设}]-> 深化综合改革试点{name: 深化综合改革试点}',
  '深圳{name: 深圳} -[relationship:{relationship: 推动}]-> 建设深港合作新引擎{name: 建设深港合作新引擎}',
  '深圳{name: 深圳} -[relationship:{relationship: 支持}]-> 提升创新体系整体效能{name: 提升创新体系整体效能}',
  '深圳{name: 深圳} -[relationship:{relationship: 高起点培育}]-> 20个战略性产业集群{name: 20个战略性产业集群}',
  '深圳{name: 深圳} -[relationship:{relationship: 推动}]-> 建设深圳湾超级总部基地、香蜜湖新金融中心等重点片区{name: 建设深圳湾超级总部基地、香蜜湖新金融中心等重点片区}',
  '深圳{name: 深圳} -[relationship:{relationship: 建设}]-> 先行示范区{name: 先行示范区} <-[relationship:{rela

In [11]:
%%ngql
USE gov_graph1;
MATCH (p:`entity`)-[:relationship]->(e:`entity`)
  WHERE p.`entity`.`name` == '五年来'
RETURN e.`entity`.`name`;

INFO:nebula3.logger:Get connection to ('127.0.0.1', 9669)


,e.entity.name
0,“粤菜师傅”“广东技工”“南粤家政”三项工程培训893万人次
1,人民群众获得感幸福感安全感进一步提升
2,坚定践行以人民为中心的发展思想
3,坚持“小切口大变化”办好民生实事
4,城镇新增就业690万人、占全国1/10
5,居民人均可支配收入超过4.5万元、增速与经济增长基本同步
6,新增基础教育公办学位252万个
7,职业教育在校生规模居全国前列
8,高等教育毛入学率提高至57%以上
9,过去五年


In [44]:
%%ngql
USE gov_graph1;
MATCH (p:`entity`)-[:relationship]->(e:`entity`)
  WHERE p.`entity`.`name` == '放管服改革'
RETURN e.`entity`.`name`;

INFO:nebula3.logger:Get connection to ('127.0.0.1', 9669)


,e.entity.name
0,政府部门权责清单、行政许可事项清单制度


In [12]:
# import QueryBundle
from llama_index import QueryBundle

# import NodeWithScore
from llama_index.schema import NodeWithScore

# Retrievers
from llama_index.retrievers import BaseRetriever, VectorIndexRetriever, KGTableRetriever

from typing import List, Optional

logger = logging.getLogger(__name__)


UNION = "union"
KG_FIRST = "kg_first"
CROSS_IF_EMPTY = "cross_if_empty"

CROSS_CHECK_PROMPT_TEMPLATE = """
You are a fact checker, now I will put a piece of context and a question, and you will check step by step on whether it's actually related or not, responding only "Yes" or "No".

For example, the context that's only partially related but actually there are details that could tell it should be uncorrelated, respond No
Do not add explanations, apologies, or any other things than Yes or No

Example:
In this example, although 保温杯 is related to 保温 in some sense, the question is not about 杯, thus from reasonable justification, it's NOT related.

context:
---
保温杯是冬天外出必备良品
---
question:
---
保温大棚是什么？
---
related:
No

Now check this with reasonable justification!

context:
---
{context}
---
question:
---
{question_str}
---
related:
"""

CROSS_CHECK_PROMPT_TEMPLATE_CHATGLM = """

你是一个事实核查员，现在我会提供一段背景和一个问题，然后你将逐步检查它们是否相关，并只回答"Yes"，表示大概率是相关的、或"No"。

例如，在这个例子中，虽然保温杯从某种意义上与保温有关，但问题并不涉及杯子，因此从这个不合理性得知它们实际上不相关。

context:
---
保温杯是冬天外出必备良品
---
question:
---
保温大棚是什么？
---
related:
No

现在开始仔细检查，通过合理性验证判断是否真正相关

context:
---
{context}
---
question:
---
{query_str}
---
related:

"""


class KGVectorCrosscheckRetriever(BaseRetriever):
    """Retriever that performs both Vector search and Knowledge Graph search, and cross-checks to mitigate hallucination"""

    def __init__(
        self,
        vector_retriever: VectorIndexRetriever,
        kg_retriever: KGTableRetriever,
        mode: str = UNION,
        cross_check_propmpt_template: str = CROSS_CHECK_PROMPT_TEMPLATE_CHATGLM,
        service_context: Optional[ServiceContext] = None,
    ) -> None:
        """Init params."""

        self._vector_retriever = vector_retriever
        self._kg_retriever = kg_retriever
        if mode not in (UNION, KG_FIRST, CROSS_IF_EMPTY):
            raise ValueError("Invalid mode.")
        self._mode = mode
        self._service_context = service_context or ServiceContext.from_defaults()
        self._cross_check_prompt_template = cross_check_propmpt_template

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        """Retrieve nodes given query."""

        kg_nodes = self._kg_retriever.retrieve(query_bundle)
        if self._mode == KG_FIRST and len(kg_nodes) > 0:
            # bypass KG retrieval
            vector_nodes = []
        else:
            vector_nodes = self._vector_retriever.retrieve(query_bundle)

        vector_ids = {n.node.node_id for n in vector_nodes}
        kg_ids = {n.node.node_id for n in kg_nodes}

        combined_dict = {n.node.node_id: n for n in vector_nodes}
        combined_dict.update({n.node.node_id: n for n in kg_nodes})

        # no matter UNION, KG_FIRST or CROSS_IF_EMPTY, we need to union them first
        retrieve_ids = vector_ids.union(kg_ids)
        # In case CROSS_IF_EMPTY and one of the retrieval got nothing, do fact-check
        # to avoid hallucinations
        one_of_the_retrieval_failed = len(retrieve_ids) > 0 and (
            len(vector_ids) == 0 or len(kg_ids) == 0)
        if self._mode == CROSS_IF_EMPTY and one_of_the_retrieval_failed:
            retrieve_ids_copy = retrieve_ids.copy()
            for node_id in retrieve_ids_copy:
                node = combined_dict[node_id]
                response = self._service_context.llm_predictor.predict(
                    self._cross_check_prompt_template,
                    context=node.node.get_content(),
                    query_str=query_bundle,
                )

                if "yes" not in str(response).lower():
                    logger.info(f"debug hallucination chunk detected, will be removed.\n Chunk: {combined_dict[node_id].node.get_text()}")
                    retrieve_ids.remove(node_id)


        retrieve_nodes = [combined_dict[rid] for rid in retrieve_ids]
        return retrieve_nodes


from llama_index import get_response_synthesizer
from llama_index.query_engine import RetrieverQueryEngine

# option a
# create raw retrievers from index
# vector_retriever = VectorIndexRetriever(index=vector_index)
# kg_retriever = KGTableRetriever(
#     index=kg_index,
#     retriever_mode="keyword",
#     include_text=False, 
#     max_keywords_per_query=3,
# )

# option b from query engine
vector_retriever = vector_query_engine._retriever
kg_retriever = kg_keyword_query_engine._retriever

combined_retriever = KGVectorCrosscheckRetriever(vector_retriever, kg_retriever, mode="cross_if_empty")

# create a response synthesizer
# response_synthesizer = get_response_synthesizer(
#     service_context=service_context,
# )
response_synthesizer = vector_query_engine._response_synthesizer or kg_keyword_query_engine._response_synthesizer

graph_vector_rag_query_engine = RetrieverQueryEngine(
    retriever=combined_retriever,
    response_synthesizer=response_synthesizer,
)

In [19]:
response = graph_vector_rag_query_engine.query("请总结过去五年来的工作，中文回答")
display(Markdown(f"<b>{response}</b>"))

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

<b>广东省在过去五年内取得了显著的工作成果。习近平总书记对广东发展给予了高度评价和亲切关怀，并多次亲自视察指导。在省委坚强领导下，广东省深入落实党中央和国务院的决策部署，坚决打赢脱贫攻坚战，成功全面建成小康社会。以下为过去五年来的工作总结：

1. 经济发展：广东省经济总量稳步增长，五年间地区生产总值破8.2万，年均增长7.5%。
2. 脱贫攻坚：广东省已实现贫困地区全部摘帽，贫困发生率降至0.1%，成功攻克了脱贫攻坚这场硬仗。
3. 产业结构调整：广东省积极推进产业结构调整，推动制造业高质量发展，重点发展战略性新兴产业，如电子信息、生物医药、新材料等，取得显著成效。
4. 教育改革：广东省加大对教育事业的投入，推动教育公平和质量提升，五年间实现了全省义务教育巩固率全国排名前移1位，实现了全省高考人数全国排名前移3位。
5. 城镇新增就业和城镇居民人均可支配收入：广东省城镇新增就业和城镇居民人均可支配收入均保持了较快速度的增长，分别年均增长690万人和4.5万元。
6. 人民群众获得感幸福感安全感：广东省全力推进民生建设，如提高粤菜师傅技能、实施南粤家政工程等，成功提升人民群众的获得感、幸福感和安全感。
7. 以人民为中心的发展思想：广东省坚定践行以人民为中心的发展思想，全力推进高质量发展，积极解决人民群众关心的问题，不断增进人民群众的福祉。</b>

In [15]:
# response = graph_vector_rag_query_engine.query("请总结有关深圳的工作，详细回答")
display(Markdown(f"<b>{response}</b>"))

<b>深圳是中国的一座重要城市，近年来在科技创新、经济发展和城市建设等方面取得了显著成就，已成为国内外知名的现代化城市。以下是深圳的一些亮点：

1. 科技创新：深圳是中国科技创新的重要引擎，拥有众多的高科技企业，如华为、腾讯等。深圳市政府鼓励企业创新，制定了一系列政策和计划，如“双创”政策，鼓励大学生、科技人才创业。

2. 经济发展：深圳是中国经济最发达的城市之一，拥有发达的制造业、服务业和金融业。深圳市政府实施了“请示式”立法，推进了司法体制改革，提高了法治水平。

3. 城市建设：深圳城市建设迅速，尤其是绿化和公共交通。深圳市政府实施了“海绵城市”计划，将城市绿化和防洪排涝相结合，提高了城市环境质量。

4. 文化氛围：深圳是一个充满活力的文化城市，有着丰富的文化活动和演出。深圳市政府鼓励文化创意产业的发展，成为了中国音乐、电影、出版的重要基地。

5. 教育：深圳市政府注重教育，提高了教育的质量和公平。深圳有一所国家级大学——深圳大学，还有多个职业技术学院和中小学。

6. 医疗：深圳市政府注重医疗健康产业的发展，提高了医疗服务的质量和覆盖面。深圳有多个医疗机构，如深圳市中心医院等。

7. 对外开放：深圳市政府积极拥抱对外开放政策，提高了深圳的国际化程度。深圳有着完善的进出口机制，成为了中国与世界交流的重要门户。

8. 城市建设管理：深圳市政府鼓励城市文化建设，优化了城市布局和交通组织，提高了城市环境质量。同时，深圳市政府也积极推动绿色发展，将污染控制和资源利用相结合，成为了中国绿色发展的重要典范。

总之，深圳是一个充满活力和魅力的城市，在经济、文化、教育、医疗等方面都取得了显著成就。未来，深圳市政府将继续努力，推动深圳朝着更加美好的方向发展。</b>